In [1]:
import requests
import pandas as pd
import numpy as np
from collections import OrderedDict
import time
from tqdm import tqdm 
import time
import requests
from bs4 import BeautifulSoup as bs

# Get all Tickers for S&P 500

In [2]:
def generate_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        tickers.append(ticker)
        
    return tickers

# Get Ticker Data from Alpha Vantage API

In [3]:
# Get ticker data from alpha vantage api
tickers = sorted(generate_sp500_tickers())

# Get API key from file
api_file = 'api_key.txt'
api_key = ''
with open(api_file) as f:
    api_key = f.readline()

for ticker in tqdm(tickers):
    try:
        # Create API call URL
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}'
        url = url.format(ticker, api_key)

        # Make API call
        r = requests.get(url)
        r.json()

        # Create OrderedDict out of data
        price_dict = OrderedDict(r.json()['Time Series (Daily)'])
        start_time = time.time()

        # Keys aren't naturally date-sorted. Do that here.
        s_list = list(price_dict.keys())
        s_list.sort()
        sorted_dict = OrderedDict()
        for key in s_list:
            sorted_dict[key] = price_dict[key]

        # Create dataframe from dictionary
        df = pd.DataFrame.from_dict(sorted_dict, orient='index')
        df = df.reindex(sorted(df.columns), axis=1)

        # Rename, then reorder columns
        df.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'div_amount', 'split_coeff']
        df = df.loc[:, ['open', 'high', 'low', 'close', 'adj_close', 'volume']]

        # Save ticker data to csv
        filename = '{}_extended.csv'.format(ticker)
        data_dir = './data/'
        df.to_csv(data_dir + filename)

        # Can only make 5 calls per min, so if more than 5 tickers, sleep
        if len(tickers) > 5:
            time.sleep(15)
            
    except KeyError:
        wait_time = time.time() - start_time
        print("API called too soon. Waiting {:.1f} seconds before querying again.".format(wait_time))
        time.sleep(wait_time)
        

 13%|█▎        | 66/505 [18:35<2:06:32, 17.30s/it]

API called too soon. Waiting 16.6 seconds before querying again.


 85%|████████▍ | 428/505 [2:00:10<21:16, 16.58s/it]  

API called too soon. Waiting 15.2 seconds before querying again.


 85%|████████▍ | 429/505 [2:00:26<20:34, 16.24s/it]

API called too soon. Waiting 30.6 seconds before querying again.


 85%|████████▌ | 430/505 [2:00:56<25:45, 20.60s/it]

API called too soon. Waiting 61.4 seconds before querying again.


KeyboardInterrupt: 

# How to Read from CSVs

In [ ]:
df = pd.read_csv('./data/aapl_extended.csv', index_col=0)
df